In [1]:
from utils import make_env


env = make_env()
env.reset()

_ = env.render("rgb_array")
_ = env.render("print")

env.close()
del env

pygame 2.4.0 (SDL 2.26.4, Python 3.10.9)
Hello from the pygame community. https://www.pygame.org/contribute.html
life: 	1	score: 	0	fps: 	0
-----------------
-               -
-               -
-               -
-               -
-               -
-               -
-               -
-       >       -
-               -
-               -
-               -
-               -
-               -
-               -
-       F       -
-----------------


In [2]:
from stable_baselines3.common.env_util import make_vec_env


vec_env = make_vec_env(lambda: make_env(1000), n_envs=8)

In [3]:
from stable_baselines3 import PPO


n_steps = 5_000_000

model = PPO("MlpPolicy", vec_env, verbose=1, tensorboard_log=f"logs/{n_steps: _}")

Using cuda device


In [4]:
model.learn(total_timesteps=n_steps, log_interval=1)

model.save("models/stablebaselines/Snake PPO")

Logging to logs/ 5_000_000\PPO_1
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 739      |
|    ep_rew_mean     | 2.63     |
| time/              |          |
|    fps             | 752      |
|    iterations      | 1        |
|    time_elapsed    | 21       |
|    total_timesteps | 16384    |
---------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 694        |
|    ep_rew_mean          | 2.76       |
| time/                   |            |
|    fps                  | 559        |
|    iterations           | 2          |
|    time_elapsed         | 58         |
|    total_timesteps      | 32768      |
| train/                  |            |
|    approx_kl            | 0.01364282 |
|    clip_fraction        | 0.139      |
|    clip_range           | 0.2        |
|    entropy_loss         | -1.6       |
|    explained_variance   | 0.126      |
|    learnin

KeyboardInterrupt: 

In [8]:
import time


vec_env.close()
del vec_env

env = make_env()

model = PPO.load("models/stablebaselines/Snake PPO")

state = env.reset()
rewards = 0
done = False

frames = []
start = time.time()
for _ in range(5000):
    action, _states = model.predict(state)
    state, reward, done, info = env.step(action)

    rewards += reward
    print("\r{}, {}".format(rewards, reward), end="")

    obs = env.render("rgb_array")
    frames.append(obs)

    if done:
        break

fps = len(frames) / (time.time() - start)
print(f"\nFPS: {fps}")

0, 0
FPS: 172.4442429168419


In [9]:
import PIL

import os


image_path = os.path.join("rl videos", f"snake_ppo_stablebaselines_{n_steps}_steps.gif")

frame_images = []
for frame in frames:
    frame_images.append(PIL.Image.fromarray(frame))

frame_images[0].save(image_path, format='GIF',
                     append_images=frame_images[1:],
                     save_all=True,
                     duration=30,
                     loop=0)